In [2]:
# !pip install python-keycloak

In [1]:
from keycloak import KeycloakOpenID

## Configure client

In [47]:
keycloak_openid = KeycloakOpenID(server_url="http://localhost:8080/auth/",
                                 client_id="rolap-web",
                                 realm_name="ROLAP",
                                 client_secret_key="CN4jMjmop2SdIZaEU5LW0Xla2wMM94Nt")

keycloak_openid

## Get Well Known

Here we can read metadata and available keyclock endpoints.
Compatible with OpenID standard.

In [48]:
config_well_known = keycloak_openid.well_known()

config_well_known

{'issuer': 'http://localhost:8080/auth/realms/ROLAP',
 'authorization_endpoint': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/auth',
 'token_endpoint': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/token',
 'introspection_endpoint': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/token/introspect',
 'userinfo_endpoint': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/userinfo',
 'end_session_endpoint': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/logout',
 'frontchannel_logout_session_supported': True,
 'frontchannel_logout_supported': True,
 'jwks_uri': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/certs',
 'check_session_iframe': 'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/login-status-iframe.html',
 'grant_types_supported': ['authorization_code',
  'implicit',
  'refresh_token',
  'password',
  'client_credentials',
  'urn:ietf:params:oauth:grant-type:

## OAuth authorization request

In [49]:
auth_url = keycloak_openid.auth_url(
    redirect_uri="http://localhost:4200/callback",
    scope="email",
    state="your_state_info")

auth_url

'http://localhost:8080/auth/realms/ROLAP/protocol/openid-connect/auth?client_id=rolap-web&response_type=code&redirect_uri=http://localhost:4200/callback&scope=email&state=your_state_info'

## Get auth url and extract the code

In [50]:
import requests
from bs4 import BeautifulSoup
response = requests.get(auth_url)

soup = BeautifulSoup(response.content, "html")
action = soup.find('form').get('action')
print(action)

http://localhost:8080/auth/realms/ROLAP/login-actions/authenticate?session_code=fVZR1h9_kLaMg9V6tnzj0XxA_CfKEqTzd8AAJCOZPYY&execution=4101924f-2f9f-4f32-afb8-e5aa8628ece0&client_id=rolap-web&tab_id=ovC-WfFZpmI


In [51]:
credentials_payload = { 'username': 'alex', 'password': 'Matematyka1!'}

session = requests.Session()
session.max_redirects = 1
response2 = session.post(action, cookies=response.cookies, data=credentials_payload, allow_redirects=False)

redirect_url = response2.headers['Location']

In [52]:
!pip install urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [53]:
import  urllib.parse
parsed_redirect_url =  urllib.parse.urlparse(redirect_url)
code = captured_value =  urllib.parse.parse_qs(parsed_redirect_url.query)['code'][0]
code

'46de227e-098f-4ec7-8d38-5375d246c9a7.2d1af13a-ba3b-4ed2-890a-acae4a065921.bced60e5-ab19-4e8d-b259-9fd07d81025c'

## Get access token with code

In [54]:
access_token = keycloak_openid.token(
    grant_type='authorization_code',
    code=code,
    redirect_uri="http://localhost:4200/callback")

access_token['access_token']

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJJendtSm1JRVk3N1EybVVwUWpReWtwWWR6Q1h3M3FzN0dyNHlIMzBIX3NjIn0.eyJleHAiOjE2ODU0NjMyNDQsImlhdCI6MTY4NTQ1NzI0NCwiYXV0aF90aW1lIjoxNjg1NDU3MjQyLCJqdGkiOiI5YzkwYjJhZC03Mzc1LTQ5MGMtOWFkYS1jZGUxMGQ4ZmY1YTgiLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjgwODAvYXV0aC9yZWFsbXMvUk9MQVAiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiODNiMTc3NTAtZDU2NC00MzE1LTkwZjEtMDJmM2QzN2UzZTI3IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoicm9sYXAtd2ViIiwic2Vzc2lvbl9zdGF0ZSI6IjJkMWFmMTNhLWJhM2ItNGVkMi04OTBhLWFjYWU0YTA2NTkyMSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiKiJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1yb2xhcCIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJzaWQiOiIyZDFhZjEzYS1iYTNiLTRlZDItODkwYS1hY2FlNGEwNjU5MjEiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwibmFtZSI6IkFsZWtzYW5kcmEgTGV3YW5kb3dza2EiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhbGV